# OptiFlex - Model Training Example

For setup, we specifyour working directory and import the necessary modules

In [13]:
import os
path = "/Users/eiddeuil/Desktop/WaveMoto" 
os.chdir(path)
print("Current Working Directory " , os.getcwd())

Current Working Directory  /Users/eiddeuil/Desktop/WaveMoto


In [5]:
import sys
import datetime
from models.DeepPoseKit import stacked_densenet
from models.DeepLabCut import deeplabcut
from models.LEAP import LEAP
from models.FlexibleBaseline import deconv_resnet_model, deconv_resnet_model_small, deconv_resnet_model_reduced
from utils.eval_func import grp_eval_pck
from training.train_func import train_generator
from training.post_train import save_record, inference, get_joint_list, print_pck

Using TensorFlow backend.
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

### 1. Folder Structure

For the purpose of this example, we setup the `dataset` and `model` folders inside our current `example` folder. In practice, user should setup `dataset` and `model` folders in the same directory as `OptiFlex` code folder.

### 2. Specify Folder Locations and Training Hyperparameters

In [7]:
# Dataset Info
dataset_animal = "fruitfly"
data_view = ""    # sd or btm only when animal is mouse

# Model Info
model_name = "LEAP"
baseline_version = ""  # baseline version only apply to FlexibleBaseline

# Input Folder Info
train_img_folder = "examples/dataset/trn_set_" + dataset_animal + "/img/" + data_view
train_lbl_folder = "examples/dataset/trn_set_" + dataset_animal + "/lbl/" + data_view
ver_img_folder = "examples/dataset/vld_set_" + dataset_animal + "/img/" + data_view
ver_lbl_folder = "examples/dataset/vld_set_" + dataset_animal + "/lbl/" + data_view
test_img_folder = "examples/dataset/tst_set_" + dataset_animal + "/img/" + data_view
test_lbl_folder = "examples/dataset/tst_set_" + dataset_animal + "/lbl/" + data_view
frame_count = 40
max_val = 16

# Output Folder Info
dataset_folder = "examples/dataset/"
pred_js_folder = dataset_folder + "pred_js_lbl"
pred_hm_folder = dataset_folder + "pred_hm_lbl"

# Training Hyperparameters
batch_size = 10
epochs = 1
learning_rate = 0.0001

### 3. Automated Variable Initialization

There are some parameters that can be interpreted from previously specified information about dataset and model, so the following code will initialize some variables base on those interpretations

In [8]:
cur_time = datetime.datetime.now().strftime("%I:%M%p-%B-%d-%Y")
dataset_name = dataset_animal   # only changed when the animal is mouse
joint_list = get_joint_list(dataset_animal)


# configuration based on dataset
if dataset_animal == "mouse":

    dataset_name = dataset_name + "_" + data_view
    num_joints = 8
    img_width = 512
    img_height = 128

elif dataset_animal == "monkey":

    num_joints = 7
    img_width = 512
    img_height = 256

elif dataset_animal == "fruitfly":

    num_joints = 32
    img_width = 256
    img_height = 256

elif dataset_animal == "zebrafish":

    num_joints = 12
    img_width = 512
    img_height = 256

else:
    print("Dataset for " + dataset_animal + " is not currently available.")
    sys.exit("Please choose dataset animal from mouse, monkey, fruitfly, or zebrafish")

# configuration based on model
if model_name == "StackedDenseNet":

    num_stage = 3
    model = stacked_densenet(img_width, img_height, num_joints, learning_rate, 2)


elif model_name == "DeepLabCut":

    num_stage = 2
    model = deeplabcut(img_width, img_height, num_joints, learning_rate)

elif model_name == "LEAP":

    num_stage = 1
    model = LEAP(img_width, img_height, num_joints, learning_rate)

elif model_name == "FlexibleBaseline":

    model_name = baseline_version + model_name

    if baseline_version == "Standard":
        num_stage = 2
        model = deconv_resnet_model(img_width, img_height, num_joints, learning_rate)
    elif baseline_version == "Reduced":
        num_stage = 2
        model = deconv_resnet_model_reduced(img_width, img_height, num_joints, learning_rate)
    elif baseline_version == "Small":
        num_stage = 1
        model = deconv_resnet_model_small(img_width, img_height, num_joints, learning_rate)
    else:
        sys.exit(baseline_version+" not supported")

else:
    print(model_name + " is not currently a supported model.")
    sys.exit("We only support LEAP, StackedDenseNet, DeepLabCut, or FlexibleBaseline")

model_filename = "_".join([model_name, dataset_name, str(frame_count)+"frame", str(epochs)+"ep",
                           str(learning_rate)+"lr", str(batch_size)+"bs", cur_time]) + ".model"



Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 256, 256, 64)      1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 256, 256, 64)      36928     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 256, 256, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 128, 128, 128)     73856     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 128, 128, 128)     147

### 4. Train Model

In [ ]:
model, history = train_generator(train_img_folder, train_lbl_folder, ver_img_folder, ver_lbl_folder,
                                 model, num_stage, batch_size, epochs, max_val)

### 5. Save Model and Training History

In [ ]:
save_record(model, history, model_filename)